<a href="https://colab.research.google.com/github/yuneeeee/PJ/blob/main/pj_2_3%EC%B0%A8%EC%8B%9C%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
# 시각화
import matplotlib.pyplot as plt 
import seaborn as sns
# 경고문구 생략
import warnings
warnings.filterwarnings('ignore')

In [ ]:
hotel_df = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/tripadviser_review.csv")
# rating = 별점 , text = 리뷰내용
hotel_df.head()

In [ ]:
# df의 정보확인
print(f'데이터 크기\n{hotel_df.shape}\n\n결측치\n{hotel_df.isnull().sum()}\n')
print(f'rating 컬럼의 값 정보\n{hotel_df.rating.value_counts()}')
print('제일 긴 리뷰 글자 수:' ,max(len(i) for i in hotel_df.text)) # 제일 긴 리뷰
print('제일 짧은 리뷰 글자 수:',min(len(i) for i in hotel_df.text)) # 제일 짧은 리뷰

In [ ]:
print(hotel_df['text'].nunique())
hotel_df.drop_duplicates(subset=['text'], inplace=True) # 중복 제거
print(f'중복데이터 제거 후 데이터 크기: {hotel_df.shape}')

In [ ]:
# 정규식을 통해 데이터 정리
hotel_df['text'] = hotel_df['text'].str.replace('[^ ㄱ-ㅣ 가-힣]','') 
hotel_df[:5]

In [ ]:
hotel_df['text'].replace('', np.nan, inplace=True)
print(len(hotel_df)) 
print(hotel_df.isnull().sum()) 
hotel_df = hotel_df.dropna(how='any')
print(f'정제한 데이터의 결측값 제거 후 데이터 크기: {hotel_df.shape}')

In [ ]:
# 한글 형태소 분석기 konlpy 다운로드
# !pip install konlpy==0.5.2 jpype1 Jpype1-py3

In [ ]:
# text 컬럼의 단어들로 말주머니 만들기
hotel_corpus = "".join(hotel_df['text'])

# Okt로 형태소 추출

In [ ]:
# 명사 형태소 추출
from konlpy.tag import Okt
from collections import Counter

In [ ]:
okt = Okt()  # 명사 형태소 추출 함수
hotel_corpusnouns_okt = okt.nouns(hotel_corpus)

In [ ]:
counter = Counter(hotel_corpusnouns_okt)

In [ ]:
counter.most_common(10)

In [ ]:
available_counter = Counter({x: counter[x] for x in counter if len(x) > 1})
available_counter.most_common(10)

# 불용어 사전

In [ ]:
# RANKS NL에 제공해주는 한국어 불용어 사전
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

In [ ]:
jeju_hotel_stopwords = ['제주', '제주도', '호텔', '리뷰', '숙소', '여행', '트립','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
for word in jeju_hotel_stopwords:
    stopwords.append(word)

# BoW 벡터 생성 후 TF-IDF 변환

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    # 정규 표현식 처리
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  
    result = hangul.sub('', text)
    # 명사 형태소 추출
    okt = Okt()  
    nouns = okt.nouns(result)
    # 한글자 키워드 제거
    nouns = [x for x in nouns if len(x) > 1]  
    # 불용어 제거
    nouns = [x for x in nouns if x not in stopwords]  
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(hotel_df['text'].tolist())
# word_list = vect.get_feature_names() # 명사 형태소
# count_list = bow_vect.toarray().sum(axis=0) # 각 단어가 전체 리뷰중에 등장한 총 횟수

In [ ]:
# BoW -> TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [ ]:
#  mapping
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')

In [ ]:
voca_size = len(invert_index_vectorizer)

# 감정분류 모델
LogisticRegression

In [ ]:
# 긍정 부정 이 두가지 감정으로 분석하기위해 rating을 이진변수로 변환

def rating_to_label(rating):
    if rating > 3:
        return 1
    else:
        return 0
    
hotel_df['label'] = hotel_df['rating'].apply(lambda x: rating_to_label(x))

In [ ]:
hotel_df.head()

In [ ]:
hotel_df['label'].value_counts()

# 긍정이 726 부정이 275개로 비율이 맞지 않음


In [ ]:
# 비율 조정
pos_random_idx = hotel_df[hotel_df['label']==1].sample(275, random_state=12).index.tolist()
neg_random_idx = hotel_df[hotel_df['label']==0].sample(275, random_state=12).index.tolist()

In [ ]:
# 데이터셋 나누기
from sklearn.model_selection import train_test_split

random_idx = pos_random_idx + neg_random_idx

X = tf_idf_vect[random_idx]
y = hotel_df['label'][random_idx]
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    random_state=333)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
# LogisticRegression로 학습 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LogisticRegression(random_state = 0)
lr.fit(X_train, y_train)

# predict in test set
y_pred = lr.predict(X_test)

# 결과
print('accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('precision: %.2f' % precision_score(y_test, y_pred))
print('recall: %.2f' % recall_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))

In [ ]:
# confusion_matrix로 시각화
from sklearn.metrics import confusion_matrix

confu = confusion_matrix(y_true = y_test, y_pred = y_pred)

plt.figure(figsize=(4, 3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# 각 단어의 coef를 시각화
lr.coef_
plt.figure(figsize=(10, 8))
plt.bar(range(len(lr.coef_[0])), lr.coef_[0])
# 양의계수 단어들은 긍정에 영향을 끼침

In [ ]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = False)

In [ ]:
# 긍정에 영향을 준 단어들
for coef in coef_pos_index[:5]:
    print(invert_index_vectorizer[coef[1]], coef[0])

In [ ]:
# 부정에 영향을 준 단어 
for coef in coef_neg_index[:5]:
    print(invert_index_vectorizer[coef[1]], coef[0])